In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB

In [2]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re
import os

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np

def majority_vote(csv_files, output_file):
    # Initialize an empty DataFrame to store the results
    result_df = pd.DataFrame(columns=['id', 'label'])

    # Iterate over each CSV file
    for csv_file in csv_files:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file)

        # Rename the 'label' column to differentiate between files
        df.rename(columns={'label': f'label_{csv_files.index(csv_file)}'}, inplace=True)

        # Merge the DataFrame with the result DataFrame
        result_df = pd.merge(result_df, df, on='id', how='outer')
    # Compute the majority vote

    result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]

    # Replace labels other than 'hyp1' and 'hyp2' with NaN
    result_df.loc[~result_df['label'].isin(['hyp1', 'hyp2']), 'label'] = np.nan

    df_temp = result_df.drop(columns=['id'])

    for index, row in result_df.iterrows():
      if pd.isna(row['label']):
        values = df_temp.iloc[index].dropna().values
        if len(values) == 0:
          result_df.at[index, 'label'] = ""
        else:
          result_df.at[index, 'label'] = values[0]


    # Write the result to a new CSV file
    result_df[['id', 'label']].to_csv(output_file, index=False)



## Paraphrasing Task Files

## ACHTUNG: muss noch für Cross Model Evaluation angepasst werden!!

## Cross model en

In [5]:
# Example usage:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_lama3_cross_model_en_v1.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_gpt4_en_v2_cross_model_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_gpt35_en_v2_cross_model_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_gemma_en_v1_cross_model.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/majority_vote_cross_model_result_en.csv'
majority_vote(csv_files, output_file)

## Cross model se

In [6]:
# Example usage:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_lama3_cross_model_se_v1.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_gpt4_se_v2_cross_model_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_gpt35_se_v2_cross_model_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_gemma_se_v1_cross_model.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/majority_vote_cross_model_result_se.csv'
majority_vote(csv_files, output_file)

<ipython-input-4-e6db54cb7eeb>:20: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]


In [ ]:
# Example usage:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_llama3_en_v1_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt4_en_v2_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt_en_narjes_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_en_vnarjes.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/majority_vote_result_en_narjes.csv'
majority_vote(csv_files, output_file)

### Voting narjes prompt

**English**

In [ ]:
# Example usage:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_llama3_prompt_narjes_en_v1_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt4_en_v2_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt_en_narjes_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_en_vnarjes.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/majority_vote_result_en_narjes.csv'
majority_vote(csv_files, output_file)

<ipython-input-4-54d2ee239503>:19: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]


**Swedish**

In [ ]:
# Example usage:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_llama3_prompt_narjes_se_v2_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt4_se_v1_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt_se_narjes_detection.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_se_vnarjes.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/majority_vote_result_se_narjes.csv'
majority_vote(csv_files, output_file)

## Translation Task Files

In [ ]:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_de_en_final.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt4/results_gpt4_de_en.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_de_en.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_de_en_final.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/majority_vote_result_de_en.csv'
majority_vote(csv_files, output_file)

<ipython-input-9-54d2ee239503>:19: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]


In [ ]:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_en_de_final.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt4/results_gpt4_en_de.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_en_de.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_en_de_final.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/majority_vote_result_en_de.csv'
majority_vote(csv_files, output_file)

<ipython-input-9-54d2ee239503>:19: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]


In [ ]:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_en_fr_final.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt4/results_gpt4_en_fr.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_en_fr.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_en_fr_final.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/majority_vote_result_en_fr.csv'
majority_vote(csv_files, output_file)

<ipython-input-9-54d2ee239503>:19: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]


In [ ]:
csv_files = ['/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_fr_en_final.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt4/results_gpt4_fr_en.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_fr_en.csv',
             '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_fr_en_final.csv']
output_file = '/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/majority_vote_result_fr_en.csv'
majority_vote(csv_files, output_file)

<ipython-input-23-e6db54cb7eeb>:20: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  result_df['label'] = result_df.iloc[:, 1:].mode(axis=1)[0]
